In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv ../.env
from ldapauthenticator.tests.conftest import get_authenticator, getenvvar
import os
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
authenticator = get_authenticator()

In [ ]:
username, userdn = authenticator.resolve_username(getenvvar("TEST_USER"))
username, userdn

In [ ]:
authenticator.use_lookup_dn_user_for_group_lookup = True

In [ ]:
ft = authenticator.authenticate(handler=None, data=dict(username=getenvvar("TEST_USER"), password=getenvvar("TEST_PASSWORD")))
ft.result()

In [ ]:
import ldap3
username, userdn = authenticator.resolve_username(getenvvar("TEST_USER"))

conn = authenticator.get_connection(getenvvar("LDAP_SEARCH_USER"), getenvvar("LDAP_SEARCH_PASSWORD"))
group = getenvvar("TEST_GROUP")
found = False
group_filter = (
    "(|"
    "(member={userdn})"
    "(uniqueMember={userdn})"
    "(memberUid={uid})"
    ")"
)
group_filter = group_filter.format(userdn=userdn, uid=username)
group_attributes = ["member", "uniqueMember", "memberUid"]
found = conn.search(
    group,
    search_scope=ldap3.BASE,
    search_filter=group_filter,
    attributes=group_attributes,
)
print(found)